<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Background" data-toc-modified-id="Background-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Background</a></span></li><li><span><a href="#Process" data-toc-modified-id="Process-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Process</a></span><ul class="toc-item"><li><span><a href="#Mapping-Eccentricity" data-toc-modified-id="Mapping-Eccentricity-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Mapping Eccentricity</a></span></li><li><span><a href="#ROI-Masking-Based-on-Eccentricity" data-toc-modified-id="ROI-Masking-Based-on-Eccentricity-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>ROI Masking Based on Eccentricity</a></span></li><li><span><a href="#Accuracy-Calculation" data-toc-modified-id="Accuracy-Calculation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Accuracy Calculation</a></span></li></ul></li><li><span><a href="#Data" data-toc-modified-id="Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data</a></span><ul class="toc-item"><li><span><a href="#All-Subject-Data" data-toc-modified-id="All-Subject-Data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>All Subject Data</a></span></li><li><span><a href="#Plots" data-toc-modified-id="Plots-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Plots</a></span></li></ul></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Analysis</a></span></li></ul></div>

# Background

---

While our eye has a wide range of vision, the 1.5mm wide fovea is by far the most sensitive area, where fine detail and color may be distinguished ([Wikipedia](https://en.wikipedia.org/wiki/Fovea_centralis)). 

While the central fovea in the image below is shown to cover 5 degrees, we found general agreement that the foveal region could be assumed to cover 2 degrees - roughy the size of a thumb held at arms length (Fairchild, 2013).

---

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/4/4c/Peripheral_vision.svg/499px-Peripheral_vision.svg.png'>

---

Our project aimed to look at areas of the visual regions that responded to foveal and non-foveal stimuli, and examine how these areas, when analyzed on their own, might predict the scene categories we used in the experiment (faces, objects, scenes, scrambled).

Fairchild, M. D. (2013). Color Appearance Models. John Wiley & Sons.

# Process

---

## Mapping Eccentricity

Explanation of calculation of PRF/eccentricities to come...



## ROI Masking Based on Eccentricity

---

In AFNI we used `3dcalc` to separate voxels representing the two eccentricity categories of interest.

To create a mask isolating the eccentricities of 2 degrees and below we ran the following code:

```afni
3dcalc -a ../masks/0003_V1_clean_mask+orig -b 0003_PRFresults_bucket+orig[6] -expr 'step(a) * step(b)*step(2-b)' -prefix 0003_V1_fov_mask
```

Where `a` is the original ROI mask and `b` is the eccentricity results in degrees for the whole brain. The `step` function then used the math `2-b` to remove any voxels from the original ROI mask which are greater than 2.

The opposite math, `b-2` was used to isolate the non-foveal eccentricities (angles greater than 2 degrees), with the following code:

```afni
3dcalc -a ../masks/0003_V1_clean_mask+orig -b 0003_PRFresults_bucket+orig[6] -expr 'step(a) * step(b)*step(b-2)' -prefix 0003_V1_non_fov_mask
```

## Accuracy Calculation

---

Accuracies were calculated in Matlab.

For classification we used `libsvm`, and the raw data, as in a previous assignment this combination had given the highest classification accuracy.

We ran the following code in Matlab in order to obtain classfication accuracy and a confusion matrix:

```matlab
dsetsV1_fov = loadROIdata('0003','V1_fov','raw');

dsetsV1_fov(1).a.categories

[accV1_fov, cmV1_fov]=classifyFPO(dsetsV1_fov);

accV1_fov
cmV1_fov

dV1_fov = diag(cmV1_fov);
dV1_fov
```

# Data

---

Below is a summary of all data. 

---

Import Libraries

In [23]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff

In [24]:
plotly.tools.set_credentials_file(username='dapper2733', api_key='sbnoWN1I8CGjjK5MzXae')

## All Subject Data

In [26]:
df = pd.read_csv("http://www.danieljwilson.com/data/Accuracies.csv")

table = ff.create_table(df)
#py.iplot(table, filename='jupyter/table1')

plotly.offline.iplot(table)

## Plots

**STILL WORKING ON GETTING THIS UP AND RUNNING...COMING SOON!**

Ignore this section for now

In [30]:
x = df.Region[(df.Subject==2) & (df.Foveal==1) & (df.Category=='Face')]
y = df.Accuracy[(df.Subject==2) & (df.Foveal==1) & (df.Category=='Face')]

In [38]:
from plotly.graph_objs import *

data = [Bar(x=df.Region[(df.Subject==2) & (df.Foveal==1) & (df.Category=='Face')],
            y=df.Accuracy[(df.Subject==2) & (df.Foveal==1) & (df.Category=='Face')])]

plotly.offline.iplot(data)

In [ ]:
Subject = 2     # {2, 3, 4}
Region = 'V1'   # {'V1', 'V2', 'V3', 'V4', }
Foveal = 1      #

In [ ]:
trace_foveal = Bar(x=df.School,
                  y=df.Women,
                  name='Foveal',
                  marker=dict(color='#ffcdd2'))

trace_non_foveal = Bar(x=df.School,
                y=df.Men,
                name='Non-Foveal',
                marker=dict(color='#A2D5F2'))

trace_gap = Bar(x=df.School,
                y=df.Gap,
                name='Difference',
                marker=dict(color='#59606D'))

data = [trace_foveal, trace_non_foveal, trace_gap]
layout = Layout(title="Average Earnings for Graduates",
                xaxis=dict(title='School'),
                yaxis=dict(title='Salary (in thousands)'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [22]:

import plotly
from plotly.graph_objs import Scatter, Layout

plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [Scatter(x=['Foveal', 'Non-Foveal'], y=[4, 3])],
    "layout": Layout(title="V1")
})

# Analysis